---

_You are currently looking at **version 1.5** of this notebook. To download notebooks and datafiles, as well as get help on Jupyter notebooks in the Coursera platform, visit the [Jupyter Notebook FAQ](https://www.coursera.org/learn/python-data-analysis/resources/0dhYG) course resource._

---

# Assignment 3 - More Pandas
This assignment requires more individual learning then the last one did - you are encouraged to check out the [pandas documentation](http://pandas.pydata.org/pandas-docs/stable/) to find functions or methods you might not have used yet, or ask questions on [Stack Overflow](http://stackoverflow.com/) and tag them as pandas and python related. And of course, the discussion forums are open for interaction with your peers and the course staff.

### Question 1 (20%)
Load the energy data from the file `Energy Indicators.xls`, which is a list of indicators of [energy supply and renewable electricity production](Energy%20Indicators.xls) from the [United Nations](http://unstats.un.org/unsd/environment/excel_file_tables/2013/Energy%20Indicators.xls) for the year 2013, and should be put into a DataFrame with the variable name of **energy**.

Keep in mind that this is an Excel file, and not a comma separated values file. Also, make sure to exclude the footer and header information from the datafile. The first two columns are unneccessary, so you should get rid of them, and you should change the column labels so that the columns are:

`['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']`

Convert `Energy Supply` to gigajoules (there are 1,000,000 gigajoules in a petajoule). For all countries which have missing data (e.g. data with "...") make sure this is reflected as `np.NaN` values.

Rename the following list of countries (for use in later questions):

```"Republic of Korea": "South Korea",
"United States of America": "United States",
"United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
"China, Hong Kong Special Administrative Region": "Hong Kong"```

There are also several countries with numbers and/or parenthesis in their name. Be sure to remove these, 

e.g. 

`'Bolivia (Plurinational State of)'` should be `'Bolivia'`, 

`'Switzerland17'` should be `'Switzerland'`.

<br>

Next, load the GDP data from the file `world_bank.csv`, which is a csv containing countries' GDP from 1960 to 2015 from [World Bank](http://data.worldbank.org/indicator/NY.GDP.MKTP.CD). Call this DataFrame **GDP**. 

Make sure to skip the header, and rename the following list of countries:

```"Korea, Rep.": "South Korea", 
"Iran, Islamic Rep.": "Iran",
"Hong Kong SAR, China": "Hong Kong"```

<br>

Finally, load the [Sciamgo Journal and Country Rank data for Energy Engineering and Power Technology](http://www.scimagojr.com/countryrank.php?category=2102) from the file `scimagojr-3.xlsx`, which ranks countries based on their journal contributions in the aforementioned area. Call this DataFrame **ScimEn**.

Join the three datasets: GDP, Energy, and ScimEn into a new dataset (using the intersection of country names). Use only the last 10 years (2006-2015) of GDP data and only the top 15 countries by Scimagojr 'Rank' (Rank 1 through 15). 

The index of this DataFrame should be the name of the country, and the columns should be ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations',
       'Citations per document', 'H index', 'Energy Supply',
       'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008',
       '2009', '2010', '2011', '2012', '2013', '2014', '2015'].

*This function should return a DataFrame with 20 columns and 15 entries.*

In [2]:
import pandas as pd
import numpy as np
import re

def answer_one():
    energy = pd.read_excel("Energy Indicators.xls", skiprows=17, skip_footer=38, parse_cols=[1, 3, 4, 5], na_values={'Unnamed: 0': np.nan})
    # 设置新的列头
    energy.columns = ['Country', 'Energy Supply', 'Energy Supply per Capita', '% Renewable']

    # 把...换成np.nan
    # 并把中间两列数据转成float64类型
    energy.replace('...', np.nan, inplace=True)
    energy.astype({'Energy Supply': 'float64', 'Energy Supply per Capita': 'float64'})

    # 把Energy Supply转成和Energy Supply per Capita同样的单位gigajoules
    energy['Energy Supply'] = energy['Energy Supply'] * 1000000

    # 把国家名替换成标准格式
    new_country = {
        "Republic of Korea": "South Korea",
        "United States of America": "United States",
        "United Kingdom of Great Britain and Northern Ireland": "United Kingdom",
        "China, Hong Kong Special Administrative Region": "Hong Kong",
        "Bolivia (Plurinational State of)": "Bolivia",
        "Iran (Islamic Republic of)": "Iran"
    }
    energy.replace({'Country': new_country}, inplace=True)

    # 把国家名中含数字的去掉数字
    energy.replace({'Country': {re.compile(r'\d+'): ''}}, inplace=True)
    energy.head()

    GDP = pd.read_csv("world_bank.csv", skiprows=4)

    # 把国家名字替换成标准格式
    new_country1 = {
        "Korea, Rep.": "South Korea", 
        "Iran, Islamic Rep.": "Iran",
        "Hong Kong SAR, China": "Hong Kong"
    }
    GDP.replace({'Country Name': new_country1}, inplace=True)
    GDP.head()

    # 获取ScimEn数据框
    ScimEn = pd.read_excel('scimagojr-3.xlsx')
    ScimEn.head()

    # 获取需要用到的GDP中的列
    gdp_columns = list(GDP.columns[0:4]) + list(GDP.columns[-10:])

    # 合并GDP和energy，得到result_df0
    result_df0 = pd.merge(left=GDP[gdp_columns], right=energy, left_on='Country Name', right_on='Country', how='inner')

    # 合并result_df0和ScimEn，得到result_df
    result_df = pd.merge(left=result_df0, right=ScimEn.loc[0:14], left_on='Country Name', right_on='Country', how='inner')

    # 把国家名列设置索引
    result_df = result_df.set_index('Country Name')
    # 获取结果需要用到的列
    result_columns = ['Rank', 'Documents', 'Citable documents', 'Citations', 'Self-citations', 'Citations per document', 'H index', 'Energy Supply', 'Energy Supply per Capita', '% Renewable', '2006', '2007', '2008', '2009', '2010', '2011', '2012', '2013', '2014', '2015']
    result_df = result_df[result_columns].sort_values(by='Rank')
    
    return result_df


### Question 2 (6.6%)
The previous question joined three datasets then reduced this to just the top 15 entries. When you joined the datasets, but before you reduced this to the top 15 items, how many entries did you lose?

*This function should return a single number.*

In [3]:
%%HTML
<svg width="800" height="300">
  <circle cx="150" cy="180" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="blue" />
  <circle cx="200" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="red" />
  <circle cx="100" cy="100" r="80" fill-opacity="0.2" stroke="black" stroke-width="2" fill="green" />
  <line x1="150" y1="125" x2="300" y2="150" stroke="black" stroke-width="2" fill="black" stroke-dasharray="5,3"/>
  <text  x="300" y="165" font-family="Verdana" font-size="35">Everything but this!</text>
</svg>

In [4]:
def answer_two():
    return len(energy) + len(GDP) + len(ScimEn) - len(result_df)

<br>

## Answer the following questions in the context of only the top 15 countries by Scimagojr Rank (aka the DataFrame returned by `answer_one()`)

### Question 3 (6.6%)
What is the average GDP over the last 10 years for each country? (exclude missing values from this calculation.)

*This function should return a Series named `avgGDP` with 15 countries and their average GDP sorted in descending order.*

In [5]:
def answer_three():
    Top15 = answer_one()
    avgGDP = Top15.iloc[0:14, 10:20].replace(np.nan, 0).apply(np.average, axis=1)
    return avgGDP

### Question 4 (6.6%)
By how much had the GDP changed over the 10 year span for the country with the 6th largest average GDP?

*This function should return a single number.*

In [6]:
def answer_four():
    Top15 = answer_one()
    return np.ptp(Top15[Top15.index.isin(avgTop6)].iloc[:, 10:20].apply(np.sum))

### Question 5 (6.6%)
What is the mean `Energy Supply per Capita`?

*This function should return a single number.*

In [7]:
def answer_five():
    Top15 = answer_one()
    return np.mean(Top15['Energy Supply per Capita'])

### Question 6 (6.6%)
What country has the maximum % Renewable and what is the percentage?

*This function should return a tuple with the name of the country and the percentage.*

In [8]:
def answer_six():
    Top15 = answer_one()
    country = Top15.sort_values(by='% Renewable', ascending=False).index[0]
    percentage = Top15.loc[country, '% Renewable']
    return (country, percentage)

### Question 7 (6.6%)
Create a new column that is the ratio of Self-Citations to Total Citations. 
What is the maximum value for this new column, and what country has the highest ratio?

*This function should return a tuple with the name of the country and the ratio.*

In [9]:
def answer_seven():
    Top15 = answer_one()
    result_df['ratio'] = result_df['Self-citations'] / result_df['Citations']
    country = result_df.sort_values(by='ratio', ascending=False).index[0]
    ratio = result_df.loc[country, 'ratio']

    return (country, ratio)

### Question 8 (6.6%)

Create a column that estimates the population using Energy Supply and Energy Supply per capita. 
What is the third most populous country according to this estimate?

*This function should return a single string value.*

In [10]:
def answer_eight():
    Top15 = answer_one()
    Top15['estimate_populous'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    country = Top15.sort_values(by='estimate_populous', ascending=False).index[2]
    return country

### Question 9 (6.6%)
Create a column that estimates the number of citable documents per person. 
What is the correlation between the number of citable documents per capita and the energy supply per capita? Use the `.corr()` method, (Pearson's correlation).

*This function should return a single number.*

*(Optional: Use the built-in function `plot9()` to visualize the relationship between Energy Supply per Capita vs. Citable docs per Capita)*

In [11]:
def answer_nine():
    Top15 = answer_one()
    result_df['Citable documents per person'] = result_df['Citable documents'] / result_df['estimate_populous']
    return result_df.loc[:,['Citable documents per person', 'Energy Supply per Capita']].corr().iat[0, 1]

In [12]:
def plot9():
    import matplotlib as plt
    %matplotlib inline
    
    Top15 = answer_one()
    Top15['PopEst'] = Top15['Energy Supply'] / Top15['Energy Supply per Capita']
    Top15['Citable docs per Capita'] = Top15['Citable documents'] / Top15['PopEst']
    #Top15.plot(x='Citable docs per Capita', y='Energy Supply per Capita', kind='scatter', xlim=[0, 0.0006])
    print(Top15)

In [13]:
#plot9() # Be sure to comment out plot9() before submitting the assignment!
# Top15 = answer_one()
plot9()

                    Rank  Documents  Citable documents  Citations  \
Country Name                                                        
China                  1     127050             126767     597237   
United States          2      96661              94747     792274   
Japan                  3      30504              30287     223024   
United Kingdom         4      20944              20357     206091   
Russian Federation     5      18534              18301      34266   
Canada                 6      17899              17620     215003   
Germany                7      17027              16831     140566   
India                  8      15005              14841     128763   
France                 9      13153              12973     130632   
South Korea           10      11983              11923     114675   
Italy                 11      10964              10794     111850   
Spain                 12       9428               9330     123336   
Iran                  13       889

### Question 10 (6.6%)
Create a new column with a 1 if the country's % Renewable value is at or above the median for all countries in the top 15, and a 0 if the country's % Renewable value is below the median.

*This function should return a series named `HighRenew` whose index is the country name sorted in ascending order of rank.*

In [14]:
def answer_ten():
    Top15 = answer_one()
    Top15['HighRenew'] = Top15['% Renewable'].apply(lambda x: 1 if x > Top15['% Renewable'].mean() else 0)
    return Top15.HighRenew

### Question 11 (6.6%)
Use the following dictionary to group the Countries by Continent, then create a dateframe that displays the sample size (the number of countries in each continent bin), and the sum, mean, and std deviation for the estimated population of each country.

```python
ContinentDict  = {'China':'Asia', 
                  'United States':'North America', 
                  'Japan':'Asia', 
                  'United Kingdom':'Europe', 
                  'Russian Federation':'Europe', 
                  'Canada':'North America', 
                  'Germany':'Europe', 
                  'India':'Asia',
                  'France':'Europe', 
                  'South Korea':'Asia', 
                  'Italy':'Europe', 
                  'Spain':'Europe', 
                  'Iran':'Asia',
                  'Australia':'Australia', 
                  'Brazil':'South America'}
```

*This function should return a DataFrame with index named Continent `['Asia', 'Australia', 'Europe', 'North America', 'South America']` and columns `['size', 'sum', 'mean', 'std']`*

In [15]:
def answer_eleven():
    Top15 = answer_one()
    temp_df = pd.DataFrame({"Country": list(ContinentDict.keys()), "Continent": list(ContinentDict.values())})

    temp_df['estimate_populous'] = temp_df['Country'].apply(lambda x: Top15.loc[x]['estimate_populous'])

    df = temp_df.groupby('Continent')\
        .agg([np.count_nonzero, np.sum, np.mean, np.std]) \
        .rename(\
                columns={'count_nonzero': 'size', 'sum': 'sum', 'mean': 'mean', 'std': 'std'})
    df.columns.names = ['first', 'second']

    Continent = df.xs('estimate_populous', level='first', axis=1, drop_level=True)
    return Continent

### Question 12 (6.6%)
Cut % Renewable into 5 bins. Group Top15 by the Continent, as well as these new % Renewable bins. How many countries are in each of these groups?

*This function should return a __Series__ with a MultiIndex of `Continent`, then the bins for `% Renewable`. Do not include groups with no countries.*

In [16]:
def answer_twelve():
    Top15 = answer_one()
    temp_df = pd.DataFrame({"Country": list(ContinentDict.keys()), "Continent": list(ContinentDict.values())})
    temp_df['% Renewable'] = temp_df['Country'].apply(lambda x: Top15.loc[x]['% Renewable'])
    temp_df['bins'] = pd.cut(temp_df['% Renewable'], 5)
    result_series = temp_df.groupby(['Continent', 'bins'])['bins'].apply(np.count_nonzero)
    return result_series

### Question 13 (6.6%)
Convert the Population Estimate series to a string with thousands separator (using commas). Do not round the results.

e.g. 317615384.61538464 -> 317,615,384.61538464

*This function should return a Series `PopEst` whose index is the country name and whose values are the population estimate string.*

In [17]:
def answer_thirteen():
    Top15 = answer_one()
    str_populous = Top15['estimate_populous'].apply(lambda x: "{:,}".format(x))
    return str_populous

### Optional

Use the built in function `plot_optional()` to see an example visualization.

In [18]:
def plot_optional():
    import matplotlib as plt
    %matplotlib inline
    Top15 = answer_one()
    ax = Top15.plot(x='Rank', y='% Renewable', kind='scatter', 
                    c=['#e41a1c','#377eb8','#e41a1c','#4daf4a','#4daf4a','#377eb8','#4daf4a','#e41a1c',
                       '#4daf4a','#e41a1c','#4daf4a','#4daf4a','#e41a1c','#dede00','#ff7f00'], 
                    xticks=range(1,16), s=6*Top15['2014']/10**10, alpha=.75, figsize=[16,6]);

    for i, txt in enumerate(Top15.index):
        ax.annotate(txt, [Top15['Rank'][i], Top15['% Renewable'][i]], ha='center')

    print("This is an example of a visualization that can be created to help understand the data. \
This is a bubble chart showing % Renewable vs. Rank. The size of the bubble corresponds to the countries' \
2014 GDP, and the color corresponds to the continent.")

In [19]:
#plot_optional() # Be sure to comment out plot_optional() before submitting the assignment!